# Hierarchy RL

[Learning Multi-Level Hierarchy with Hindsight](#Learning-Multi-Level-Hierarchy-with-Hindsight)  
[Hindsight Experience Replay](#Hindsight-Experience-Replay)  

## Learning Multi-Level Hierarchy with Hindsight 

[Link](https://arxiv.org/pdf/1712.00948.pdf), [Code](https://github.com/nikhilbarhate99/Hierarchical-Actor-Critic-HAC-PyTorch), [Open Review](https://openreview.net/forum?id=ryzECoAcY7)  

This work combines the universal value function approximator (UVFA) and Hindsight Experience Replay (HER).  
UVFA will be used to estimate the action-value function of a goal-conditioned policy $\pi, q_\pi(s, g, a)$.   
HER is a data augmentation technique that can accelerate learning in sparse reward tasks. 

### Technique Details
#### Space
The state spaces of all layers of the hierarchy is identical.  
The action space of bottom-most layer is identical to the original action space of the task, other layer has the space that is identical to the state space. 

#### Nested Policy
Policy at layer $i$ generate the goal of layer $i-1$.  

#### Hindsight Action Transitions
Similar to [Hindsight Experience Replay](#Hindsight-Experience-Replay).  

#### Subgoal Test Transitions
To my understanding, this is just a random simulation see whether the hindsight goal can be achieved or not, but their claim and explanation is not very clear to me.  

## Hindsight Experience Replay
[link](https://arxiv.org/pdf/1707.01495.pdf)  

Handle the challenge caused by binary, sparse reward.  
By replaying the experience with different goal (no additional simulation needed), more informative reward can be generated. But this approach is different with reward shaping, because it does not requires any domain knowledge. 

Prerequired knowledge: Universal Value Function Approximators (UVFA) and how replay buffer.   
Read the example in **section 3.1** and **algorithm 1**, one can easily get how this simple approach works.   
The $r_g$ is defined in the section 4.1. 